In [28]:
import sqlite3 as sq
import pandas as pd
import numpy as np
from pandas import DataFrame as df
import pprint
# from interval import Interval as intt
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [3]:
# 分数预估
conn = sq.connect('高考数据库.sqlite3')     # 连接数据库
c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type='table'")     # 读取表名
table = c.fetchall()
table
cates = table[10][0]     # 获取类名
mmajors = table[14][0]     # 获取专业表名
colleges = table[15][0]     # 获取学校名
province = table[12][0]
rankings = table[13][0]
fir = table[11][0]

# 获取表的列名、列名结构
def col(form):
    c.execute("SELECT * FROM {}".format(form))
    col_name = [tuple[0] for tuple in c.description]
    
    c.execute("PRAGMA table_info({})".format(form))
    col_structure = c.fetchall()
    
    return col_name,col_structure

# 获取表的内容（DF结构）
def f_df(form):
    sql = "SELECT * FROM {}".format(form)
    values = c.execute(sql)
    f_df = pd.DataFrame(data = values)
    return f_df

In [15]:
# 河南一分一档表
rankings21 = f_df(rankings)
rankings21.columns = col(rankings)[0]
rankings21 = rankings21[rankings21['year'] == 2021]
rankings21 = rankings21[rankings21['provinceID_id'] == 1]
colleges21 = f_df(colleges)


In [16]:
# 专业学校推荐
alldf = pd.read_csv('alldata1.0.csv',encoding='utf-8',index_col=0)

def rec0(cate,score,prov,c_rank,major,m_rank,risk):
    
    # 筛选文理科
    df = alldf[alldf['categoryID_id'] == cate]     
    
    # 河南2021一分一档对照表
    rec_rank = rankings21[rankings21['categoryID_id'] == cate]     
    ranking = rec_rank[rec_rank['score'] == score]['rank'].values[0]
    print(ranking)
    
    # 筛选专业
    df = df[df['c1'] == major[0]]
    if major[1] != None:
        df = df[df['c2'] == major[1]]
            
    # 统计综合得分
    for provID in df['provinceLoc']:
        if provID == prov:
            df.loc[df['provinceLoc'] == prov,'sum_score'] = (df['cityScore']*(1 + c_rank/10) + df['majorScore']*m_rank) / (c_rank + m_rank)
        else:
            df.loc[df['provinceLoc'] != prov,'sum_score'] = (df['cityScore']*1 + df['majorScore']*m_rank) / (c_rank + m_rank)
    
    # 学校代码转换学校名称
    for dfs in df['collegeID_id']:
        df.loc[df['collegeID_id'] == dfs,'college'] = colleges21[colleges21[0] == dfs][1].values[0]
    df = df.sort_values(['preRanking'],ascending=True)

    # 报考排名范围：0.9ranking-1.5ranking
    if risk == 1:     # 输出所有符合条件的推荐方案
        df_temp1 = df[df['preRanking'] >= ranking*0.9]
        df_temp2 = df_temp1[df_temp1['preRanking'] < ranking*1.0]
        df_temp3 = df[df['preRanking'] >= ranking*1.0][0:6]
        df = pd.concat([df_temp2,df_temp3],axis=0 ,ignore_index=True)
        if df.empty:
            df = df[df['preRanking'] >= ranking*1.0][0:6]
    elif risk ==2:     # 输出报考风险相对大的推荐方案
#         df = df[df['preRanking'] < ranking*1.0]    
        df = df[df['preRanking'] >= ranking*0.9]
        df = df[0:6]
    else:     # 输出报考稳妥的推荐方案 
#         df = df[df['preRanking'] < ranking*1.5]    
        df = df[df['preRanking'] >= ranking*1.0]
        df = df[0:6]
        
        
    df = df.sort_values(['sum_score'],ascending=False)
    df = df.loc[:,('provinceLoc','cityLoc','college','majorName','preRanking','sum_score')]
    
    return df
   

In [35]:
def rec(cate,score,prov,c_rank,majors,m_rank,risk):
    '''
    输入为：文理科，高考分数，期望省份，地域重要性，期望专业（多选），专业重要性，风险性
    输出为：该推荐大学+专业的省市位置，大学名，专业名，预计所需最低排名，推荐指数
    '''
    
    # 筛选文理科
    df = alldf[alldf['categoryID_id'] == cate]     
    
    # 河南2021一分一档对照表
    rec_rank = rankings21[rankings21['categoryID_id'] == cate]     
    if score>=rec_rank.describe()['score']['max']:
        ranking=rec_rank.describe()['rank']['min']
    else:
        ranking = rec_rank[rec_rank['score'] == score]['rank'].values[0]
    print(ranking)
    
    df_pass=pd.DataFrame()
    
    for major in majors:
        # 筛选专业
        if major[1]==None:#输入 c1 None
            df_temp = df[df['c1'].str.contains(major[0])]
            df_pass = pd.concat([df_pass,df_temp],axis=0 ,ignore_index=True)
        else:#输入c1 c2
            df_temp = df[df['c2'].str.contains(major[1])]
            df_temp1 = df[(df['c1'].str.contains(major[0])) & (df['c2']=='')]
            df_pass = pd.concat([df_pass,df_temp,df_temp1],axis=0 ,ignore_index=True)
        
    df=df_pass.drop_duplicates()
    
    # 统计综合得分
    for provID in df['provinceLoc']:
        if provID == prov:
            df.loc[df['provinceLoc'] == prov,'sum_score'] = (df['cityScore']*(1 + c_rank/10) + df['majorScore']*m_rank) / (c_rank + m_rank)
        else:
            df.loc[df['provinceLoc'] != prov,'sum_score'] = (df['cityScore']*1 + df['majorScore']*m_rank) / (c_rank + m_rank)
            
    
    # 学校代码转换学校名称
    for dfs in df['collegeID_id']:
        df.loc[df['collegeID_id'] == dfs,'college'] = colleges21[colleges21[0] == dfs][1].values[0]
    df = df.sort_values(['preRanking'],ascending=True)

    # 报考排名范围：0.9ranking-1.5ranking
    if risk == 1:     # 输出所有符合条件的推荐方案
        df_temp1 = df[df['preRanking'] >= ranking*0.9]
        df_temp2 = df_temp1[df_temp1['preRanking'] < ranking*1.0]
        df_temp3 = df[df['preRanking'] >= ranking*1.0][0:6]
        df = pd.concat([df_temp2,df_temp3],axis=0 ,ignore_index=True)
        df = df[0:12]
        if df.empty:
            df = df[df['preRanking'] >= ranking*1.0][0:6]
    elif risk ==2:     # 输出报考风险相对大的推荐方案
#         df = df[df['preRanking'] < ranking*1.0]    
        df = df[df['preRanking'] >= ranking*0.9]
        df = df[0:6]
    else:     # 输出报考稳妥的推荐方案 
#         df = df[df['preRanking'] < ranking*1.5]    
        df = df[df['preRanking'] >= ranking*1.0]
        df = df[0:6]
    
    max_score=df.describe()['sum_score']['max']
    df['sum_score']=df['sum_score']/max_score*100
        
    df = df.sort_values(['sum_score'],ascending=False,ignore_index=True)
    df = df.loc[:,('provinceLoc','cityLoc','college','majorName','preRanking','sum_score')]
    
    return df

In [37]:
rec(2,675,'北京省',5,[['工学',None]],3,1)

982


,provinceLoc,cityLoc,college,majorName,preRanking,sum_score
0,北京市,北京市,北京航空航天大学,工科试验班类(航空航天类),1159,100.000000
1,北京市,北京市,北京航空航天大学,工科试验班类(医工交叉试验班),953,95.296823
2,北京市,北京市,北京航空航天大学,工科试验班类(国际通用工程学院),953,94.374631
3,上海市,上海市,同济大学,工科试验班(建筑规划景观与设计类),1272,94.144544
4,北京市,北京市,北京航空航天大学,工科试验班类(信息类),953,93.729097
5,湖北省,武汉市,华中科技大学,计算机科学与技术(启明本硕博实验班),953,91.333089
6,湖北省,武汉市,华中科技大学,计算机科学与技术(计算机卓越计划实验班),953,91.333089
7,湖北省,武汉市,华中科技大学,电气工程及其自动化(启明本硕博实验班),1272,91.333089
8,上海市,上海市,同济大学,工科试验班(信息类),1272,86.163768
9,湖北省,武汉市,华中科技大学,光电信息科学与工程(王大珩光电创新实验班),1159,85.846049


In [34]:
1/1.1

0.9090909090909091

## 用不上的函数

In [102]:
# 暂时用不上的函数
def mins(college,major):
    df = alldf[alldf['collegeID_id'] == college]
    df = df[df['c1'] == major[0]]
    df = df[df['c2'] == major[1]]
    minscore = df[df['minScore'] == min(df['minScore'])]
    return minscore

True

In [269]:
ttt = rankings21[rankings21['score'] == 750]
ttt.empty

True

In [273]:
rankings21.shape[0]

1185